# Advanced Modeling of Groundwater Flow (GW3099)
![header](../img/header.jpg)

# Exercise 7: MODFLOW 6

The purpose of this exercise is to convert the RIV and lake CHD packages to SFR and LAK packages, respectively, and add a mover package to transfer water between the SFR and LAK packages. Additionally we will also convert the SFR inflow to use timeseries input.

In [ ]:
import os
import numpy as np
import flopy
import matplotlib.pyplot as plt

In [ ]:
# load the existing model
model_ws = './ex06-data'
s = flopy.mf6.MFSimulation().load(sim_ws=model_ws)

In [ ]:
# get list of files in the ex07-data directory
ex07ddir = './ex07-data'
os.listdir(ex07ddir)

In [ ]:
# change work space
model_ws = './ex07-completed'
s.set_sim_path(model_ws)

In [ ]:
# get the groundwater model
gwf = s.get_model('ex06')

In [ ]:
# get the river location data
riv = gwf.get_package('RIV-1')
rivspd = riv.stress_period_data.get_data(key=0)
rivspd.dtype

In [ ]:
# get rid of the existing river package
gwf.remove_package('RIV')

### create the SFR model

In [ ]:
# read sfr package data
sfrpd = np.genfromtxt('./ex07-data/sfr-packagedata.dat', names=True)
sfrpd.dtype

In [ ]:
# create a default dtype
sfrpackagedata = flopy.mf6.ModflowGwfsfr.packagedata.empty(gwf, boundnames=True, maxbound=rivspd.shape[0])
sfrpackagedata.dtype

In [ ]:
# fill package data
for name in sfrpackagedata.dtype.names :
    if name in rivspd.dtype.names:
        sfrpackagedata[name] = rivspd[name]
for name in sfrpackagedata.dtype.names:
    if name in sfrpd.dtype.names:
        sfrpackagedata[name] = sfrpd[name]
sfrpackagedata['boundnames'] = rivspd['boundname']    
sfrpackagedata    

In [ ]:
type(sfrpackagedata['cellid'][0])

In [ ]:
# read sfr connection data
with open('ex07-data/sfr-connectiondata.dat') as f: 
    lines = f.readlines()
sfrconnectiondata = []
for line in lines:
    t = line.split()
    c = []
    for v in t:
        i = int(v)
        c.append(i)
    sfrconnectiondata.append(c)

In [ ]:
# create sfr inflows
sfrperioddata = {0: [[0, 'inflow', 86400.]], 1:[[0, 'inflow', 95040.]]}

In [ ]:
# add the sfr package to the model
sfr = flopy.mf6.ModflowGwfsfr(gwf, stage_filerecord='ex06.sfr.stage.bin', budget_filerecord='ex06.sfr.cbc', 
                              mover=True,
                              unit_conversion=128390.00, 
                              boundnames=True, nreaches=38,
                              packagedata=sfrpackagedata, connectiondata=sfrconnectiondata,
                              perioddata=sfrperioddata)

### Create the Lake package

In [ ]:
# get the lake chd data
lakchd = gwf.get_package('LAK-1')
chdspd = lakchd.stress_period_data.get_data(key=0)
chdloc = chdspd['cellid']
chdloc

In [ ]:
# remove the lake chd file
gwf.remove_package('LAK-1')

In [ ]:
# read lak package data
dtype = [('lakeno', np.int32), ('iconn', np.int32), 
         ('k', np.int32), ('i', np.int32), ('j', np.int32), 
         ('claktype', '|U20'), ('bedleak', np.float), ('belev', np.float), 
         ('telev', np.float), ('connlen', np.float), ('connwidth', np.float)]
lakpd = np.genfromtxt('./ex07-data/lak-connectiondata.dat', dtype=dtype)
lakpd.dtype

In [ ]:
arr = np.column_stack((lakpd['k'], lakpd['i'], lakpd['j']))
cellid = tuple(map(tuple, arr))
len(cellid)

In [ ]:
# create the lake package

### reset idomain using lak chd locations

In [ ]:
idomain = gwf.dis.idomain.array
for loc in chdloc:
    idomain[loc] = 0
gwf.dis.idomain.set_data(idomain[0], layer=0)

### Create the MVR package

In [ ]:
s.write_simulation()

In [ ]:
# add head observations
head_obs = [('W1_1', 'HEAD', (0, 15, 18)), ('W1_3', 'HEAD', (2, 15, 18)), ('W1_5', 'HEAD', (4, 15, 18))]
gwobs = flopy.mf6.ModflowUtlobs(gwf, continuous={'ex06.head.obs.csv': head_obs})

In [ ]:
# determine where the chds representing the lakes are
chd2 = gwf.get_package('LAK-1')
chd2.stress_period_data.get_data(key=0)

In [ ]:
# add chd observations
chd2obsname = 'ex06.lak.obs'
chd2.obs_filerecord.set_data([chd2obsname])
chd_obs = [('LAKE-1', 'CHD', 'lake1'), ('LAKE-2', 'CHD', 'lake2')]
chdobs = flopy.mf6.ModflowUtlobs(gwf, continuous={'ex06.lake.obs.csv': chd_obs}, parent_file=chd2, fname=chd2obsname)

In [ ]:
# determine where the rivers are
riv = gwf.get_package('RIV-1')
riv.stress_period_data.get_data(key=0)

In [ ]:
# add river observations
rivobsname = 'ex06.riv.obs'
riv.obs_filerecord.set_data([rivobsname])
riv_obs = [('RIVER-1', 'RIV', 'seg1'), ('RIVER-2', 'RIV', 'seg2'), 
           ('RIVER-3', 'RIV', 'seg3'), ('RIVER-4', 'RIV', 'seg4')]
rivobs = flopy.mf6.ModflowUtlobs(gwf, continuous={'ex06.riv.obs.csv': riv_obs}, parent_file=riv, fname=rivobsname)

In [ ]:
# write the model files to ex06-completed and run the simulation
s.write_simulation()
s.run_simulation()

In [ ]:
# retrieve the heads
hobj = flopy.utils.HeadFile(os.path.join(model_ws, 'ex06.hds'))
times= hobj.get_times()

In [ ]:
# create a spatial reference from the grb
grb = flopy.utils.MfGrdFile(os.path.join(model_ws, 'ex06.dis.grb'), )
sr = grb.get_spatialreference()
sr.get_extent()

In [ ]:
h = hobj.get_data(totim=times[0])
mm = flopy.plot.ModelMap(sr=sr)
mm.plot_array(h, masked_values=[1e+30])
c = mm.contour_array(h, masked_values=[1e+30], colors='white', levels=np.arange(30, 50, 2))
plt.clabel(c, fmt='%3d');

In [ ]:
h = hobj.get_data(totim=times[-1])
mm = flopy.plot.ModelMap(sr=sr)
mm.plot_array(h, masked_values=[1e+30])
c = mm.contour_array(h, masked_values=[1e+30], colors='white', levels=np.arange(30, 50, 2))
plt.clabel(c, fmt='%3d');

In [ ]:
# load and plot the ex06 observations
gwobs = np.genfromtxt(os.path.join(model_ws, 'ex06.head.obs.csv'), delimiter=',', names=True)
for name in gwobs.dtype.names[1:]:
    plt.semilogx(gwobs['time'], gwobs[name], marker='.', label=name)
plt.legend()

In [ ]:
# load chd and river obs
chdobs = np.genfromtxt(os.path.join(model_ws, 'ex06.lake.obs.csv'), delimiter=',', names=True)
rivobs = np.genfromtxt(os.path.join(model_ws, 'ex06.riv.obs.csv'), delimiter=',', names=True)

In [ ]:
# plot the chd lake results
for name in chdobs.dtype.names[1:]:
    plt.semilogx(chdobs['time'], chdobs[name], marker='.', label=name)
plt.legend();

In [ ]:
# plot the river results
for name in rivobs.dtype.names[1:]:
    plt.semilogx(rivobs['time'], rivobs[name], marker='.', label=name)
plt.legend();